# Comparison Between TreeValue and DM-Tree

## Run Environment Information

In [1]:
import os
import platform
import shutil

import psutil
from hbutils.scale import size_to_bytes_str

print('OS:', platform.platform())
print('Python:', platform.python_implementation(), platform.python_version())
print('Processor:', platform.processor())
print('CPUS:', os.cpu_count())
print('Memory:', size_to_bytes_str(psutil.virtual_memory().total, precision=3))
print('Has CUDA:', 'Yes' if shutil.which('nvidia-smi') else 'No')

OS: Linux-5.15.0-1021-azure-x86_64-with-glibc2.2.5
Python: CPython 3.8.14
Processor: x86_64
CPUS: 2
Memory: 6.781 GiB
Has CUDA: No


## Flatten and Unflatten

We create a dictionary for flatten

In [2]:
origin = {'a': 1, 'b': 2, 'c': {'x': 3, 'y': 4}}

### TreeValue's Performance

In [3]:
from treevalue import FastTreeValue, flatten, unflatten

origin_tree = FastTreeValue(origin)

In [4]:
flatted = flatten(origin_tree)
flatted

[(('a',), 1), (('b',), 2), (('c', 'x'), 3), (('c', 'y'), 4)]

In [5]:
%timeit flatten(origin_tree)

687 ns ± 3.23 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [6]:
unflatten(flatted)

<TreeValue 0x7f58104a6220>
├── 'a' --> 1
├── 'b' --> 2
└── 'c' --> <TreeValue 0x7f58104a6e50>
    ├── 'x' --> 3
    └── 'y' --> 4

In [7]:
%timeit unflatten(flatted)

753 ns ± 3.08 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### DM-Tree's Performance

This is the [dm-tree](https://github.com/deepmind/tree) library
And here is a simplest example of dm-tree.

In [8]:
from tree import flatten, flatten_with_path

We try to flatten `origin` with both [flatten](https://tree.readthedocs.io/en/latest/api.html#tree.flatten) and [flatten_with_path](https://tree.readthedocs.io/en/latest/api.html#tree.flatten_with_path) function, and then measure their performance.

In [9]:
flatten(origin)

[1, 2, 3, 4]

In [10]:
%timeit flatten(origin)

1.2 µs ± 49.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [11]:
flatten_with_path(origin)

[(('a',), 1), (('b',), 2), (('c', 'x'), 3), (('c', 'y'), 4)]

In [12]:
%timeit flatten_with_path(origin)

16.3 µs ± 140 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Obviously, `flatten` in dm-tree is an irreversible operation, but `flatten_with_path` is reversible, which is similar to `flatten` in treevalue.

But the [unflatten_as](https://tree.readthedocs.io/en/latest/api.html#tree.unflatten_as) is absolutely another thing in dm-tree

In [13]:
from tree import unflatten_as

In [14]:
unflatten_as({'a': None, 'b': None, 'c': {'x': None, 'y': None}}, [1, 2, 3, 4])

{'a': 1, 'b': 2, 'c': {'x': 3, 'y': 4}}

In [15]:
%timeit unflatten_as({'a': None, 'b': None, 'c': {'x': None, 'y': None}}, [1, 2, 3, 4])

13.6 µs ± 109 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Positional Replacement

This is the performance in treevalue

In [16]:
from treevalue import flatten, unflatten

def replace(t, v):
    pairs = flatten(t)
    return unflatten([(path, vi) for (path, _), vi in zip(pairs, v)])

In [17]:
replace(origin_tree, [3, 5, 7, 9])

<TreeValue 0x7f58104b0d60>
├── 'a' --> 3
├── 'b' --> 5
└── 'c' --> <TreeValue 0x7f58104b09a0>
    ├── 'x' --> 7
    └── 'y' --> 9

In [18]:
%timeit replace(origin_tree, [3, 5, 7, 9])

2.19 µs ± 19.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


This is the performance in dm-tree

In [19]:
from tree import unflatten_as

In [20]:
unflatten_as(origin, [3, 5, 7, 9])

{'a': 3, 'b': 5, 'c': {'x': 7, 'y': 9}}

In [21]:
%timeit unflatten_as(origin, [3, 5, 7, 9])

13.2 µs ± 191 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Treevalue's performance is much better.